Name: Arya Jalali

Student ID: 98105665

In this exercise, you should develop a character-level RNN language model.

You are free to choose the architecture, but you must use GRUs and not LSTMs. A linear embedding layer (hidden size 64), a 2-layer GRU (hidden size 128, dropout 0.1), and a linear classifier head is an example architecture.

You should generate some example outputs using beam search.

Some parts of the code has been done for you. You need to implement the parts that raise `NotImplementedError`.

The index zero has been reserved for the padding token/character. By subtracting one from the token indices, the indices will become ASCII indices. (And the padding index will become `-1`.)

The model's classification head should directly predict ASCII characters (256 possibilities). It should not predict any special tokens, such as padding, start or end.

# Bootstrap

# Install

In [1]:
! pip install -U torch datasets pyperclip icecream numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/

# Download the Data

In [2]:
!wget https://files.lilf.ir/Black%20Luminary.txt

--2023-05-11 18:55:22--  https://files.lilf.ir/Black%20Luminary.txt
Resolving files.lilf.ir (files.lilf.ir)... 82.102.11.148
Connecting to files.lilf.ir (files.lilf.ir)|82.102.11.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3148450 (3.0M) [text/plain]
Saving to: ‘Black Luminary.txt’

Black Luminary.txt  100%[===================>]   3.00M  3.93MB/s    in 0.8s    

2023-05-11 18:55:23 (3.93 MB/s) - ‘Black Luminary.txt’ saved [3148450/3148450]



In [3]:
! ls -lh

total 3.1M
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt'
drwxr-xr-x 1 root root 4.0K May 10 13:36  sample_data


In [4]:
! realpath *.txt

/content/Black Luminary.txt


# User Config

In [5]:
data_paths = [
    '/content/Black Luminary.txt',
    ]

## imports

In [6]:
import pyperclip

In [7]:
from icecream import ic

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
device = torch.device("cpu")
#: We will set device again in the training loop.

In [10]:
import datasets as D

In [11]:
import numpy
np = numpy

import statistics

# Utils

In [12]:
class NumpyPrintOptions:
    def __init__(self, **kwargs):
        self.options = kwargs
        self.original_options = np.get_printoptions()

    def __enter__(self):
        np.set_printoptions(**self.options)

    def __exit__(self, exc_type, exc_value, traceback):
        np.set_printoptions(**self.original_options)

class NoTruncationNumpyPrintOptions(NumpyPrintOptions):
    def __init__(self):
        super().__init__(
            threshold=np.inf, 
            linewidth=200, 
            suppress=True, 
            precision=4
        )

In [13]:
import jax

def torch_shape_get(input):
    def h_shape_get(x):
        return x.dtype, x.shape

    return jax.tree_map(h_shape_get, input)

In [14]:
def has_nan(tensor):
    return torch.any(torch.isnan(tensor))

In [15]:
class ModelEvalMode:
    def __init__(self, model):
        self.model = model

    def __enter__(self):
        self.model.eval()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.model.train()

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data

In [17]:
d = D.load_dataset("text",
                         data_files=data_paths, sample_by="paragraph")
d

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-751f099585d07f63/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18423
    })
})

In [18]:
d = d['train']
d

Dataset({
    features: ['text'],
    num_rows: 18423
})

In [19]:
d[1000:1010]

{'text': ['Professor Snape threw him backwards, and Harry stumbled, but just managed to keep standing.',
  "'I understand, sir. I apologise if my careless words have offended.'",
  "'Get going then!' The man turned around and marched towards his desk.",
  'Not keen on his company for the moment, Harry hastily made his way towards the hall.',
  "Merlin, did he have to grab my arm like that? If being implicated in murder is child's play to him, then I can indeed do without partaking in his 'problems'…",
  '~BLHD~',
  "Harry paused before entering the Great Hall and forced his countenance as hard as he could into a blank expression. He must not let up; he must not relent for one second. Weakness would not help him here. It might also be prudent to distance himself a bit from other people for a while to limit the damage to their reputation and family; depending on how the whole situation turned out, the political fallout could be immense. With a sense of foreboding, he imagined Daphne's re

In [20]:
def str_to_np(s, dtype=np.int8):
    s = s.encode('ascii', errors='ignore')
    return np.frombuffer(s, dtype=dtype)

str_to_np('hello')

array([104, 101, 108, 108, 111], dtype=int8)

In [21]:
def str_to_onehot(s):
    return np.eye(256)[str_to_np(s)]

In [22]:
dc = d.map(lambda batch: {'input': [str_to_np(t).astype(np.int32) + 1 for t in batch['text']]}, batched=True) #: added one to the char indices to make zero available for the pad token
dc

Map:   0%|          | 0/18423 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input'],
    num_rows: 18423
})

In [23]:
dc = dc.filter(lambda x: (len(x['input']) > 30 and len(x['text'].split()) > 4), batched=False)
dc

Filter:   0%|          | 0/18423 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input'],
    num_rows: 16371
})

In [24]:
dc.set_format("torch", columns=["input",])

In [25]:
dc = dc.shuffle()

In [26]:
dcs = dc.train_test_split(test_size=0.2)
dcs

DatasetDict({
    train: Dataset({
        features: ['text', 'input'],
        num_rows: 13096
    })
    test: Dataset({
        features: ['text', 'input'],
        num_rows: 3275
    })
})

# Model

- [GRU --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

- [torch.nn.utils.rnn.pack_sequence --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_sequence.html#torch.nn.utils.rnn.pack_sequence) (not necessarily needed)

- [Embedding --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

- [torch.nn.utils.rnn.pad_sequence --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html)


In [27]:
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_sequence
import torch
import torch.nn as nn


def simple_elementwise_apply(fn, packed_sequence):
    """applies a pointwise function fn to each element in packed_sequence"""
    return torch.nn.utils.rnn.PackedSequence(fn(packed_sequence.data), packed_sequence.batch_sizes)

class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.embedding = nn.Embedding(256, 64)
        self.gru = nn.GRU(64, 256, num_layers = 2, dropout = 0.1)
        self.linear = nn.Linear(256, 256)

    
    def forward(self, x, hidden=None):
        # x: list of tensors shaped (seq_length, )
        # The seq_length will not necessarily be equal for all list items!
        
        packed_seq = pack_sequence(x, enforce_sorted = False)

        embedded = simple_elementwise_apply(self.embedding, packed_seq)
        output, hidden = self.gru(embedded)
        output = simple_elementwise_apply(self.linear, output)
        
        return output.data, hidden

    def predict(self, x, h0 = None):
      x = x.unsqueeze(0)
      embedded = self.embedding(x)
      if h0 != None:
        output, hidden = self.gru(embedded)
      else:
        output, hidden = self.gru(embedded)
      output = self.linear(output)
      
      return output, hidden


A = Model()
x = [torch.zeros(5, dtype=torch.long),
     torch.zeros(8, dtype=torch.long),
     torch.zeros(3, dtype=torch.long)]

output, hidden = A(x)
output.shape

torch.Size([16, 256])

In [28]:
def loss_fn(y, y_hat):
    #: y: list of tensors shaped (seq_length)
    #: y_hat: tensor shaped (B, T, classes_n)
    #:
    #: Be sure to skip padding!

    packed_target = pack_sequence(y, enforce_sorted = False)
    loss = F.cross_entropy(y_hat,packed_target.data, reduction = 'mean')
    return loss

In [29]:
def shift_left(tensor_list, pad_value=0.0):
    shifted_tensors = []
    for tensor in tensor_list:
        shifted_tensors.append(tensor[1:])
    
    return shifted_tensors

# Example usage:
input_ids = [torch.tensor([1, 2, 3, 4], device = device), torch.tensor([5, 6, 7, 8], device = device)]
print("Input Ids:")
print(input_ids)

target_ids = shift_left(input_ids)
print("Shifted Left (Target Ids):")
print(target_ids)

Input Ids:
[tensor([1, 2, 3, 4], device='cuda:0'), tensor([5, 6, 7, 8], device='cuda:0')]
Shifted Left (Target Ids):
[tensor([2, 3, 4], device='cuda:0'), tensor([6, 7, 8], device='cuda:0')]


# Beam Search Generation

In [30]:
import torch
import heapq

def tensor_to_string(tensor):
    chars = [chr(c) for c in tensor]
    return ''.join(chars)

def tensor_append_scalar(tensor, scalar):
    scalar_tensor = torch.tensor(scalar).view(1)  # Add a dimension to match the original tensor's dimensions
    scalar_tensor = scalar_tensor.to(device)

    # Append the scalar to the original tensor
    result = torch.cat((tensor, scalar_tensor), dim=0)
    return result


def generate_next_top_k(model, input_sequence, k):
    logits, _ = model.predict(input_sequence)
    logits = logits[0, -1, :]
    # ic(torch_shape_get(logits))
    
    probabilities = torch.softmax(logits, dim=-1)
    # ic(torch_shape_get(probabilities))

    top_k_values, top_k_indices = torch.topk(probabilities, k)

    return [(tensor_append_scalar(input_sequence, idx.item() + 1), log_prob.item()) for idx, log_prob in zip(top_k_indices, top_k_values.log())]

def beam_search(model, desired_length, starting_string, k=5):
    with ModelEvalMode(model), torch.no_grad():
      input_sequence = torch.tensor(str_to_np(starting_string).astype(np.int32) + 1, dtype=torch.long)
      input_sequence = input_sequence.to(device)
      # ic(torch_shape_get(input_sequence))
      
      log_prob = 0.0

      beam = [(input_sequence, log_prob)]

      while len(beam[0][0]) < desired_length:
          new_beam = []
          for seq, log_prob in beam:
              next_top_k = generate_next_top_k(model, seq, k)
              new_beam.extend([(new_seq, new_log_prob + log_prob) for new_seq, new_log_prob in next_top_k])

          beam = heapq.nlargest(k, new_beam, key=lambda x: x[1])

      return [tensor_to_string(seq - 1) for seq, _ in beam]

In [31]:
a = str_to_np("Harry ").astype(np.int32) + 1
ic(a)
tensor_to_string(a - 1)

ic| a: array([ 73,  98, 115, 115, 122,  33], dtype=int32)


'Harry '

In [32]:
def eval_gen(*args, display=999999, **kwargs):
    generated_texts = beam_search(
        *args, **kwargs,
    )

    for idx, text in enumerate(generated_texts):
        if idx >= display:
            break
        
        print(f"Generated text {idx + 1}: {text}")

# Train

In [35]:
m = Model().to(device)
eval_gen(display=3, model=m, desired_length=100, starting_string="Harry ", k=32)

Generated text 1: Harry "á½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½ì
Generated text 2: Harry "á½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½Æ
Generated text 3: Harry "Ë½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½j1½ì


In [37]:
#: Training Loop

if torch.cuda.is_available():
	device = 'cuda'
	non_blocking = True
elif True:
	device = 'cpu'
	non_blocking = False

epochs = 31
batch_size = 32
learning_rate = 0.001

m = Model().to(device=device)
m.train()

optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

counter = 0
for epoch in range(epochs):
	if epoch == 50:
		p
	dc = dc.shuffle()
	total_loss = 0
	for i in range(0, len(dc), batch_size):
		batch = dc[i:i+batch_size]
		inputs = batch['input']
	 
		inputs = list(map(lambda x: x.to(device), inputs))

		optimizer.zero_grad()

		output, hidden = m(list(map(lambda x: x[:-1], inputs)))

		targets = shift_left(inputs)
		
		l = loss_fn(targets, output)
	
		l.backward()
		optimizer.step()
		total_loss += l.item()

		if counter % 10 == 0:
			print(f"loss: {l:>7f}  [{counter:>5d}, epoch={epoch}]")
		
		counter += 1

	print(f"loss: {total_loss * batch_size / len(dc):>7f}  [{counter-1:>5d}, epoch={epoch} finished!]")
	if epoch % 15 == 0:
		eval_gen(display=3, model=m, desired_length=100, starting_string="Harry ", k=32)

	
None

loss: 5.551413  [    0, epoch=0]
loss: 3.210406  [   10, epoch=0]
loss: 3.091456  [   20, epoch=0]
loss: 3.002703  [   30, epoch=0]
loss: 2.948586  [   40, epoch=0]
loss: 2.886783  [   50, epoch=0]
loss: 2.772266  [   60, epoch=0]
loss: 2.693014  [   70, epoch=0]
loss: 2.644928  [   80, epoch=0]
loss: 2.594368  [   90, epoch=0]
loss: 2.572421  [  100, epoch=0]
loss: 2.488661  [  110, epoch=0]
loss: 2.442610  [  120, epoch=0]
loss: 2.412068  [  130, epoch=0]
loss: 2.374264  [  140, epoch=0]
loss: 2.337102  [  150, epoch=0]
loss: 2.311154  [  160, epoch=0]
loss: 2.304539  [  170, epoch=0]
loss: 2.280418  [  180, epoch=0]
loss: 2.268611  [  190, epoch=0]
loss: 2.203117  [  200, epoch=0]
loss: 2.157763  [  210, epoch=0]
loss: 2.187656  [  220, epoch=0]
loss: 2.139466  [  230, epoch=0]
loss: 2.115527  [  240, epoch=0]
loss: 2.105806  [  250, epoch=0]
loss: 2.080462  [  260, epoch=0]
loss: 2.030721  [  270, epoch=0]
loss: 2.014584  [  280, epoch=0]
loss: 2.027045  [  290, epoch=0]
loss: 2.05

In [38]:
# m = Model()
eval_gen(display=3, model=m, desired_length=100, starting_string="Harry ", k=32)

Generated text 1: Harry Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 2: Harry Bfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 3: Harry Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjp


In [39]:
eval_gen(display=50, model=m, desired_length=250, starting_string="Harry ", k=100)

Generated text 1: Harry Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 2: Harry Bfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 3: Harry Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjp
Generated text 4: Harry Qvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj

In [40]:
eval_gen(display=50, model=m, desired_length=250, starting_string="Arcturus ", k=100)

Generated text 1: Arcturus Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 2: Arcturus Qvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 3: Arcturus Bfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 4: Arcturus Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv

In [41]:
eval_gen(display=50, model=m, desired_length=150, starting_string="Draco ", k=100)

Generated text 1: Draco Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 2: Draco Bfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 3: Draco Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjp
Generated text 4: Draco Qvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 5: Draco Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 6: Draco Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvfvjvjvjvjvjvjvjvjvjvjvjvjvjvj

In [42]:
eval_gen(display=50, model=m, desired_length=150, starting_string="Harry looked at ", k=100)

Generated text 1: Harry looked at Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 2: Harry looked at Bfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvj
Generated text 3: Harry looked at Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjp
Generated text 4: Harry looked at Qvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 5: Harry looked at Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvfvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjv
Generated text 6: Harry looked at Bvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvjvfvjvjvjvjvjvjvjvjvjvjvjvjvjvj